In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from utils import imread
from detectBlobs import detectBlobs
from drawBlobs import drawBlobs
#from skimage.color import rgb2gray
from skimage.color import rgb2gray
from scipy.ndimage.filters import convolve
from scipy.ndimage import gaussian_laplace
import sys
%matplotlib inline

In [2]:
imgs = ['butterfly.jpg', 'einstein.jpg', 'fishes.jpg', 'sunflowers.jpg']
numBlobsToDraw = 500
imageName = imgs[1]
datadir = os.path.join('..', 'data', 'blobs')
im = imread(os.path.join(datadir, imageName))

In [3]:
def py_im2double(img):
    grayscaled = rgb2gray(img)
    original_img = grayscaled.astype('float')
    img_min, img_max = np.min(original_img.ravel()), np.max(original_img.ravel())
    return (original_img - img_min) / (img_max - img_min)

def laplacian_of_gaussian_filter(sigma):
    kernel_size = np.round(6*sigma)#np.round(6*sigma)
    if kernel_size % 2 == 0:
        kernel_size+=1
    half_size=np.floor(kernel_size/2)
    x, y = np.meshgrid(np.arange(-half_size, half_size+1), np.arange(-half_size, half_size+1))
    
    exp_term=np.exp(-(x**2+y**2) / (2*sigma**2))
    exp_term[exp_term < sys.float_info.epsilon * exp_term.max()] = 0
    if exp_term.sum() != 0:
        exp_term = exp_term/exp_term.sum() 
    else: 
        exp_term
    kernel = -((x**2 + y**2 - (2*sigma**2)) / sigma**2) * exp_term 
    kernel=kernel-kernel.mean()
    return kernel

def create_scale_space(gray_image,sigma_scale_factor,initial_sigma,level):
    h,w=np.shape(gray_image)
    scale_space = np.zeros((h,w,level),np.float32)
    sigma = [0]*(level+1)
    sigma[0] = initial_sigma
    for i in range(0,level):
        print('Convolving with sigma={}'.format(sigma[i]))
        kernel=laplacian_of_gaussian_filter(sigma[i])
        convolved_image=convolve(gray_image,kernel)
        #cv2.imshow("LoG Convolved Image with sigma={}".format(sigma[i]),convolved_image)
        scale_space[:,:,i] = np.square(convolved_image)
        sigma[i+1]=sigma[i]*sigma_scale_factor
    return scale_space, sigma

In [4]:
def py_im2double(img):
    original_img = img.astype('float')
    img_min, img_max = np.min(im.ravel()), np.max(im.ravel())
    return (original_img - img_min) / (img_max - img_min)

def laplacian_scaled_space(im, scaling, sigma_seed, level):
    h, w = im.shape
    scale_space = np.zeros((h, w, level))
    sigma = np.zeros((level+1))
    sigma[0] = sigma_seed
    
    for itr in range(level):
        print ("Iterating for Sigma = {}".format(sigma[itr]))
        scale_space[:, :, itr] = gaussian_laplace(im, sigma=sigma[itr]) ** 2
        sigma[itr+1] = sigma[itr] * scaling  
    return scale_space, sigma

def max_scaled_spaces(scale_spaces, i, j, l, l1):
    h, w = scale_spaces[:, :, 0].shape
    search_over = [(0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), (1, 0), (-1, 0)]
    flag = True
    for _i, _j in search_over:
        if (i+_i in range(h)) and (j+_j in range(w)):
            if scale_spaces[i+_i, j+_j, l1] >= scale_spaces[i, j, l]:
                flag = False

#     return all(scale_spaces[i + dx, j + dy, l1] < scale_spaces[i, j, l] 
#        for dx, dy in search_over 
#        if  0<= i + dx < h and 0<= j + dy <w)
    
    return flag

def non_max_suppression_blobs(scale_spaces, scaling, sigma, level_, cutoff=0.003):
    scale_spaces_max = scale_spaces.copy()
    h, w = scale_spaces_max[:, :, 0].shape
    blob_location = []
    kernel = [int(np.ceil(s)*scaling) for s in sigma] #int(np.ceil(s)) int(np.ceil(np.sqrt(2)*sigma[i]))
    print (kernel)
    
    def check(l):
        size = np.shape(scale_spaces[:,:,0])
        search_over = [(0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1), (1, 0), (-1, 0)]
        return all(scale_spaces[i + dx, j + dy, l] < scale_spaces[i, j, le] 
           for dx, dy in search_over 
           if  0<= i + dx < size[0] and 0<= j + dy <size[1])
    
    for le in range(0, level_):
        print ("Iterating for Level={}".format(le))
        curr = kernel[le]
        scale_spaces_max[-curr:, -curr:, le] = 0
        scale_spaces_max[:curr, :curr, le] = 0
        for i in range(curr+1, (h - curr - 1)):
            for j in range(curr+1, (w - curr - 1)):
                
                if scale_spaces[i, j, le] < cutoff:
                    continue
                    
                curr_flag = max_scaled_spaces(scale_spaces, i, j, le, le)

                lower_flag = (le > 0) and scale_spaces[i, j, le-1] < scale_spaces[i, j, le] and max_scaled_spaces(scale_spaces, i, j, le, le-1)

                
#                 if le > 3 and check(le) :
#                     print ((le > 0), check(le-1) )
                
                upper_flag = (le < level_-1) and scale_spaces[i, j, le+1] < scale_spaces[i, j, le]  and max_scaled_spaces(scale_spaces, i, j, le, le+1) 
                
                #print (curr_flag, lower_flag, upper_flag)
                if curr_flag  and upper_flag: #and lower_flag
#                     if (le > 3):
#                         x1, x2, x3 = (le > 0), (scale_spaces_max[i, j, le-1] < scale_spaces_max[i, j, le]), (max_scaled_spaces(scale_spaces_max, i, j, le, le-1))
#                         m1 = x1 and x2 and x3
#                         print (lower_flag, x1, x2, x3, m1)
                    #print ("HI")
                    blob_location.append([i, j, le, scale_spaces[i, j, le]])
                    scale_spaces_max[i, j, le] = 1
        
    blobs = np.zeros((len(blob_location), 5))
    i = 0
    for item in blob_location:
        x, y = item[0], item[1]
        radius = sigma[item[2]]
        score = item[3]
        blobs[i] = [y, x, radius, -1, score] #(x, y, radius, angle, score)
        i+=1
    return blobs   

In [9]:
def detectBlobs(im, param=None):
    processed_im = py_im2double(rgb2gray(im))
    scaling = np.sqrt(1.8)
    sigma_seed = .9
    level = 14
    scale_space, sigma = create_scale_space(processed_im, scaling, sigma_seed, level)
    #create_scale_space(gray_image,sigma_scale_factor,initial_sigma,level)
    blobs = non_max_suppression_blobs(scale_space, scaling, sigma, level, .0003)
    return blobs
blob1 = detectBlobs(im)

Convolving with sigma=0.9
Convolving with sigma=1.2074767078498865
Convolving with sigma=1.62
Convolving with sigma=2.1734580741297957
Convolving with sigma=2.9160000000000004
Convolving with sigma=3.9122245334336325
Convolving with sigma=5.248800000000001
Convolving with sigma=7.042004160180539
Convolving with sigma=9.447840000000003
Convolving with sigma=12.675607488324973
Convolving with sigma=17.006112000000005
Convolving with sigma=22.81609347898495
Convolving with sigma=30.611001600000012
Convolving with sigma=41.068968262172916


MemoryError: 

In [ ]:
x1 = set(blob1[:, 2])
x1

In [ ]:
drawBlobs(im, blob1[:, [0, 1, 2, 4]], 500)

In [ ]:
1.8/np.sqrt(2)

In [ ]:
for s i

In [ ]:
c_max = check(k)
l_max = u_max = True
if k - 1 >= 0:
    l_max = check(k - 1) and \
    scale_space[i, j, k - 1] < scale_space[i, j, k]
if k + 1 < level:
    u_max = check(k + 1) and \
    scale_space[i, j, k + 1] < scale_space[i, j, k]
if c_max and l_max and u_max:
    max_scale_space[i, j, k] = 1
    blob_location.append((i, j, k, scale_space[i, j, k]))

In [ ]:
max_scale_space = scale_space
threshold_factor = .001
mask = [0] * (level)
index = [(1, 0), (-1, 0), (0, 1), (0, -1), 
         (1, 1), (1, -1), (-1, 1), (-1, -1)]
for i in range(0, level):
    mask[i]=int(np.ceil(np.sqrt(2)*sigma[i]))
size = np.shape(scale_space[:,:,0])

def check(l):
    return all(scale_space[i + dx, j + dy, l] < scale_space[i, j, k] 
       for dx, dy in index 
       if  0<= i + dx < size[0] and 0<= j + dy <size[1])

blob_location =[]
for k in range(0,level):
    max_scale_space[:mask[k],:mask[k],k] = 0
    max_scale_space[-mask[k]:,-mask[k]:,k] = 0
    
    for i in range(mask[k]+1,size[0]-mask[k]-1):
        
        for j in range(mask[k]+1,size[1]-mask[k]-1):
            
            if scale_space[i, j, k] < threshold_factor:
                continue
            #print (k)
            c_max = check(k)
            l_max = u_max = True
            if k - 1 >= 0:
                l_max = check(k - 1) and \
                scale_space[i, j, k - 1] < scale_space[i, j, k]
            if k + 1 < level:
                u_max = check(k + 1) and \
                scale_space[i, j, k + 1] < scale_space[i, j, k]
            if c_max and l_max and u_max:
                max_scale_space[i, j, k] = 1
                blob_location.append((i, j, k, scale_space[i, j, k]))

In [ ]:
def check1(l):
    print (scale_space.shape, index)
    return all(scale_space[i + dx, j + dy, l] < scale_space[i, j, k] 
       for dx, dy in index 
       if  0<= i + dx < size[0] and 0<= j + dy <size[1])

In [ ]:
check1(5)

In [ ]:
processed_im = py_im2double(rgb2gray(im))
scaling = np.sqrt(2)
sigma_seed = 1
level = 7
scale_space, sigma = laplacian_scaled_space(processed_im, scaling, sigma_seed, level)

In [ ]:
scale_space.shape

In [ ]:
def detectBlobs(im, param=None):
    # Input:
    #   IM - input image
    #
    # Ouput:
    #   BLOBS - n x 5 array with blob in each row in (x, y, radius, angle, score)
    #
    # Dummy - returns a blob at the center of the image
    processed_im = py_im2double(rgb2gray(im))
    initial_sigma = 1.4 #1.6
    k = np.sqrt(2) #initial_scale
    sigma_scale_factor = np.sqrt(1.7)
    n_iterations = 15
    level = 15 #10-15
    threshold_factor = .003
    h, w = processed_im.shape
    scale_space = np.zeros((h, w, level))
    scale_space, sigma = create_scale_space(processed_im,sigma_scale_factor, initial_sigma,level)
    max_scale_space = np.copy(scale_space)
    mask = [0] * (level)
    index = [(1, 0), (-1, 0), (0, 1), (0, -1), 
             (1, 1), (1, -1), (-1, 1), (-1, -1)]
    for i in range(0, level):
        mask[i]=int(np.ceil(np.sqrt(2)*sigma[i]))
    size = np.shape(scale_space[:,:,0])

    def check(l):
        return all(scale_space[i + dx, j + dy, l] < scale_space[i, j, k] 
           for dx, dy in index 
           if  0<= i + dx < size[0] and 0<= j + dy <size[1])

    blob_location =[]
    for k in range(0,level):
        max_scale_space[:mask[k],:mask[k],k] = 0
        max_scale_space[-mask[k]:,-mask[k]:,k] = 0
        for i in range(mask[k]+1,size[0]-mask[k]-1):
            for j in range(mask[k]+1,size[1]-mask[k]-1):
                if scale_space[i, j, k] < threshold_factor:
                    continue
                c_max = check(k)
                l_max = u_max = True
                if k - 1 >= 0:
                    l_max = check(k - 1) and \
                    scale_space[i, j, k - 1] < scale_space[i, j, k]
                if k + 1 < level:
                    u_max = check(k + 1) and \
                    scale_space[i, j, k + 1] < scale_space[i, j, k]
                if c_max and l_max and u_max:
                    max_scale_space[i, j, k] = 1
                    blob_location.append((i, j, k, scale_space[i, j, k]))

    blobs = np.zeros((len(blob_location), 5))
    i = 0
    for center in blob_location:
        x, y = center[0], center[1]
        radius = int(np.ceil(np.sqrt(2)*sigma[center[2]])) 
        score = center[3]
        blobs[i] = [y, x, radius, -1, score] #(x, y, radius, angle, score)
        i+=1
    return np.array(blobs)


In [ ]:
drawBlobs(im, blobs[:, [0, 1, 2, 4]], numBlobsToDraw)

In [ ]:
if 5 in range(0, 5):
    print ("Yes")
else: print ("NO")

In [ ]:
k1 = 0.3
for l in range(15):
    k1 *= np.sqrt(2)
    print (k1)